# Huggingface Transformers
Beginner-focused Jupyter Notebook introducing Hugging Face Transformer pipelines: Learn essentials of text embeddings, classification, question answering, named entity recognition and text generation using simple steps and pre-trained models. No prerequisites!

## What are Transformers?

Natural Language Processing is a field of linguistics and machine learning focused on understanding everything related to human language. The aim of NLP tasks is not only to understand single words individually, but to be able to understand the context of those words.

The following is a list of common NLP tasks, with some examples of each:

- Sentiment Analysis
- Question Answering
- Named Entity Recognition
- Text Summarization
- Text Generation
- Text Classification: Zero-shot, Multi-label
- Mask Filling
- Text Translation


**Transformer models** can be used to solve all kinds of NLP tasks, like the ones mentioned above. The 🤗 Transformers library provides the functionality to create and use those shared models. The Model Hub contains thousands of pretrained models that anyone can download and use. You can also upload your own models to the Hub!

**Resources:**
1.  [Huggingface Transformers](https://huggingface.co/transformers/)
2.  [Huggingface Model Hub](https://huggingface.co/models)
3.  [LLM Bootcamp](https://github.com/miztiik/llm-bootcamp)

In this notebook, we will use the `transformers` library to use pre-trained models for various NLP tasks. We will use the `pipeline` class to use pre-trained models for various NLP tasks. The `pipeline` class provides a simple API dedicated to several NLP tasks. It provides a simple, straight-forward, and efficient way to use pre-trained models.

<a href="https://colab.research.google.com/github/miztiik/llm-bootcamp/chapters/hf_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
# Comment the above line to see the installation logs

# Install the latest version of the transformers library
!pip install -qU transformers

## Pipeline
The most basic object in the 🤗 Transformers library is the `pipeline()` function. It connects a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer.

![Pipeline](images/transformer-pipeline.png)

Refer to the [Huggingface Pipelines](https://huggingface.co/docs/transformers/en/main_classes/pipelines) for more details.

In [5]:
from transformers import pipeline

### Sentiment analysis pipeline

<img src="images/sentiment_analysis.png" width="50%"/>

In [32]:
sentiment_task = pipeline("sentiment-analysis")
sentences = [
    "Dr Stone is great anime.",
    "I hate this so much!",
]
sentiment_task(sentences)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9907229542732239}]

Pipelines groups together three steps: preprocessing, passing the inputs through the model, and postprocessing:
<img src="images/sentiment_analysis_01.png" width="50%"/>

It is best practice to setup a cache directory for the transformers library. This will allow us to reuse the downloaded models and avoid downloading them again.

In [10]:
import os

# Setting the Hugging Face Model Cache Directory
MODEL_CACHE_DIR = "./../../../models_cache"

# https://stackoverflow.com/questions/63312859/how-to-change-huggingface-transformers-default-cache-directory
os.environ["HF_HOME"] = MODEL_CACHE_DIR
os.environ["TRANSFORMERS_CACHE"] = MODEL_CACHE_DIR

# os.environ["HF_DATASETS_CACHE"] = f"{MODEL_CACHE_DIR}/datasets"
# os.environ["TORCH_HOME"] = f"{MODEL_CACHE_DIR}/torch"

**Sentiment Analysis with Custom Model**

Lets choose a model from the [Model Hub: Text Classification](https://huggingface.co/models?pipeline_tag=text-classification&sort=likes) or Search for a `sentiment` analysis model https://huggingface.co/models?search=sentiment




In [16]:
# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

sentiment_model = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline(
    "sentiment-analysis", model=sentiment_model, tokenizer=sentiment_model
)
sentences = [
    "Dark Knight Rises was a great movie!.",
    "I hate this so much!",
]
sentiment_task(sentences)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'positive', 'score': 0.97898930311203},
 {'label': 'negative', 'score': 0.9291250109672546}]

### Question Answering 

QA systems differ in the way answers are created.

- **Extractive QnA**: The model extracts the answer from a context and provides it directly to the user. It is usually solved with BERT-like models.
  <img src="images/ex_q_a_01.png" width=50%>
- **Generative QnA**: The model generates free text directly based on the context. It leverages Text Generation models and provides more flexible answers.
  <img src="images/gen_q_a_01.png" width=50%>

Try Question Answering

In [18]:
qa_model = pipeline("question-answering")

question = "What is zero?"
context = "zero is a number representing an empty quantity. Adding zero to any number leaves that number unchanged.  Multiplying any number by zero has the result zero, and consequently, division by zero has no meaning in arithmetic."

qa_response = qa_model(question=question, context=context)
print(qa_response)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.8385463953018188, 'start': 8, 'end': 47, 'answer': 'a number representing an empty quantity'}


The model returns a dictionary containing the keys:

`answer`: The text extracted from the context, which should contain the answer.

`start`: The index of the character in the context that corresponds to the start of the extracted answer.

`end`: The index of the character in the context that corresponds to the end of the extracted answer.

`score`: The confidence of the model in extracting the answer from the context.

### Named Entity Recognition

Named Entity Recognition (NER) is the NLP task of identifying key information (entities) in text. An entity is a set of contiguous words that appear in the document and refers to the same thing. Some examples of entities are "Kumar", "India". Usually, entities are classified into categories like `Name`, `Country`.

  <img src="images/ner.png" width=50%>

Try NER

In [27]:
ner = pipeline("ner")
ner("What was Gandhi doing in India on 15 August 1947?")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER',
  'score': 0.9981331,
  'index': 3,
  'word': 'Gandhi',
  'start': 9,
  'end': 15},
 {'entity': 'I-LOC',
  'score': 0.99966097,
  'index': 6,
  'word': 'India',
  'start': 25,
  'end': 30}]

In [30]:
ner = pipeline("ner", aggregation_strategy="simple")
ner("What was Gandhi and Nehru doing in India on 15 August 1947?")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.9975593,
  'word': 'Gandhi',
  'start': 9,
  'end': 15},
 {'entity_group': 'PER',
  'score': 0.99828976,
  'word': 'Nehru',
  'start': 20,
  'end': 25},
 {'entity_group': 'LOC',
  'score': 0.99964774,
  'word': 'India',
  'start': 35,
  'end': 40}]

### Text Summarization

Text summarization is a task whose goal is generating a concise and precise summary of long texts, without losing the overall meaning. There are two main approaches to automatic text summarization: Abstractive summarization and Extractive summarization.

- **Extraction-based summarization**: A subset of words or sentences that represent the most important points is pulled from the long text and combined to make a summary. The results may not be grammatically accurate.

- **Abstraction-based summarization**: Advanced deep learning techniques (mainly in seq-to-seq models) are applied to paraphrase and shorten the original document, just like humans do. Since abstractive machine learning algorithms can generate new phrases and sentences that represent the most important information from the source text, they can assist in overcoming the grammatical inaccuracies of the extraction-based techniques.

<img src="images/text_summarization.png" width=50%>

Try Text Summarization

In [47]:
from transformers import pipeline

summarizer = pipeline("summarization")

text = """
The Uttaramerur inscriptions describe the distinctive Kudavolai system employed in the local governance of the Chola kingdom. In this administrative method, one representative per ward was selected via an electoral process. Contestant names were recorded on palm leaf tickets, which were subsequently placed inside a pot and mixed thoroughly. A young child drew out the tickets randomly, announcing the names of those chosen as representatives. Through this procedure, thirty individuals were elected to serve as ward administrators. The intriguing election technique came to be known as the Kudavolai system.
"""
summarizer(text, max_length=100, min_length=10)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' Uttaramerur inscriptions describe the distinctive Kudavolai system employed in the local governance of the Chola kingdom . In this administrative method, one representative per ward was selected via an electoral process . Contestant names were recorded on palm leaf tickets, which were placed inside a pot and mixed thoroughly .'}]

### Text Generation

<img src="images/text_gen.png" width=50%>

Try Text Generation

In [48]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to solve many real-world issues and learn how using real-world problems can dramatically reduce your risk of illness.\n\nThis course will provide you with the basics, along with some of the most effective'}]

You can control how many different sequences are generated with the argument `num_return_sequences` and the total length of the output text with the argument `max_length`.

In [57]:
generator("Science fiction movies are based on ",
          max_length=15, num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Science fiction movies are based on \xa0a series of novels and other novels'},
 {'generated_text': 'Science fiction movies are based on vernacular translations or, in contemporary comics'}]

### Text Classification: Zero-shot classification

**What is Zero-Shot Learning?**
Zero-Shot Learning is the ability to detect classes that the model has never seen during training. It resembles our ability as humans to generalize and identify new things without explicit supervision.

For example, let’s say we want to do `sentiment classification` and `news category` classification. Normally, we will train/fine-tune a new model for each dataset. In contrast, with zero-shot learning, you can perform tasks such as sentiment and news classification directly without any task-specific training.

<img src="images/zero-shot-vs-transfer.png" width=50%>

Try Text Summarization

In [38]:
from transformers import pipeline

txt_to_classify = "You are learning about transformer library in this notebook. It is part of a course on LLM Bootcamp."

classifier = pipeline("zero-shot-classification")
classifier(
    txt_to_classify,
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'You are learning about transformer library in this notebook. It is part of a course on LLM Bootcamp.',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.969709038734436, 0.02318527176976204, 0.007105606608092785]}

### Text Classification: Multi-label classification
If more than one candidate label can be correct, pass multi_label=True to calculate each class independently:

In [1]:
txt_to_classify = "I love travelling and enjoying the cuisines of the world."
candidate_labels = ["travel", "cooking", "dancing", "exploration"]
classifier(txt_to_classify, candidate_labels, multi_label=True)

NameError: name 'classifier' is not defined

### Mask filling

The `top_k` argument controls how many possibilities you want to be displayed. Note that here the model fills in the special `<mask>` word, which is often referred to as a _mask token_. Other mask-filling models might have different mask tokens, so it’s always good to verify the proper mask word when exploring other models. One way to check it is by looking at the mask word used in the widget.

<img src="images/zero-shot-vs-transfer.png" width=50%>


In [58]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'score': 0.1961977779865265,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052717983722687,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

### Text Translation
<img src="images/text_translation_01.jpg" width=50%>

In [63]:
%%time
from transformers import pipeline

# French to English
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator(
    "Apprendre est amusant, j'aime apprendre l'intelligence artificielle.")

CPU times: total: 328 ms
Wall time: 1.57 s


[{'translation_text': 'Learning is fun, I like learning artificial intelligence.'}]